In [ ]:
from utils.graphing_utils import *
from utils.markov_utils import *
from utils.rsm_model_utils import *
from utils.wolfram_utils import *

from concurrent.futures import ThreadPoolExecutor

import wolframclient
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
from wolframclient.serializers import export
from fractions import Fraction
import sympy as sym

In [ ]:
"""
Set up wolfram session

Get wolfram kernel path: https://reference.wolfram.com/language/WolframClientForPython/docpages/basic_usages.html
Get wolfram license: https://software.berkeley.edu/mathematica
"""

wolfram_kernel_path = get_wolfram_kernel_path()

wolfram_session_manager = WolframSessionManager(wolfram_kernel_path)

# If this fails, check your wolfram path
wolfram_session = wolfram_session_manager.start_session()
wolfram_session.evaluate(wlexpr("1 + 1"))

In [ ]:
"""
Define typical RSM failure parameters
"""

conservative_failure_params = FailureParameters(
    failure_rps=Fraction(1, 50 * 24),
    recovery_rps=Fraction(1, 5),
    human_recovery_rps=Fraction(1, 5),
    update_rps=Fraction((60 * 60 * 1000), 10),
    outdate_rps=Fraction((60 * 60 * 1000), 10),
)

bad_failure_params = FailureParameters(
    failure_rps=Fraction(1, 50 * 24) * 2,
    recovery_rps=Fraction(1, 5),
    human_recovery_rps=Fraction(1, 5),
    update_rps=Fraction((60 * 60 * 1000), 10),
    outdate_rps=Fraction((60 * 60 * 1000), 10),
)

sym_failure_params_capital = FailureParameters(
    failure_rps=sym.Symbol("B"),
    recovery_rps=sym.Symbol("R"),
    human_recovery_rps=sym.Symbol("H"),
    update_rps=sym.Symbol("U"),
    outdate_rps=sym.Symbol("O"),
)

sym_failure_params_lowercase = FailureParameters(
    failure_rps=sym.Symbol("b"),
    recovery_rps=sym.Symbol("r"),
    human_recovery_rps=sym.Symbol("h"),
    update_rps=sym.Symbol("u"),
    outdate_rps=sym.Symbol("o"),
)

In [ ]:
"""
Test generate and evaluate MTTF for a replicated state machine
"""
cmm = get_dr_backup_cmm(3, 2, conservative_failure_params, conservative_failure_params)
# cmm = get_dr_cmm(3, conservative_failure_params)
# cmm = get_dr_good_bad_cmm(2, 1, conservative_failure_params, conservative_failure_params)


wolfram_mm = get_wolfram_markov_model(cmm)
failed_wolfram_id = get_wolfram_failed_id(cmm)

round(get_mtt_state(wolfram_mm, failed_wolfram_id, wolfram_session), 2)

In [ ]:
P = wl.ContinuousMarkovProcess([1, 0, 0], 
        [
            [-2, 1, 1],
            [1, -2, 1],
            [1, 1, -2],
        ],
    )
print(get_percent_in_states_bounded(P, [1,2], 100000, wolfram_session))


get_percent_in_states_unbounded(P, [1,2], wolfram_session)


In [ ]:
state_ids

In [ ]:
cmm.state_to_id = {x:y-1 for x,y in cmm.state_to_id.items()}

In [ ]:
get_graphviz(cmm, show_weights=True, node_to_group=working_and_backup_grouping(cmm))


In [ ]:
cmm = get_dr_cmm(3, conservative_failure_params)

# cmm.state_transitions[cmm.state_to_id["Failed"]].transition_rates = [('3:0', Fraction(10**100))]
cmm.state_transitions[cmm.state_to_id["1:1"]].transition_rates[1] = ('3:0', Fraction(1,1200))
cmm.state_transitions[cmm.state_to_id["1:0"]].transition_rates[1] = ('3:0', Fraction(1, 1200))
cmm.state_to_id.pop('Failed')
cmm.state_transitions.pop(0)
cmm.state_to_id = {x:y-1 for x,y in cmm.state_to_id.items()}

state_ids = [x+1 for x in get_live_state_ids(cmm, 3)]
wolfram_cmm = get_wolfram_markov_model(cmm)

# get_percent_in_states_bounded(wolfram_cmm, state_ids, 1000 * 365 * 24, wolfram_session)
get_percent_in_states_unbounded(wolfram_cmm, state_ids, wolfram_session)

In [ ]:
0.7059895296182115391499569729586754363525694206895087024643106784826965651721843966539932846809522159985942459055757636428997437303007763968562065349916833

In [ ]:
cmm = get_dr_cmm(3, conservative_failure_params)

cmm.state_transitions[cmm.state_to_id["Failed"]].transition_rates = [('3:0', Fraction(1, 100000))]
# cmm.state_transitions[cmm.state_to_id["Failed"]].transition_rates = [('3:0', Fraction(10**5, 1))]

state_ids = [x+1 for x in get_live_state_ids(cmm, 3)]
wolfram_cmm = get_wolfram_markov_model(cmm)

convert_wolfram_number(get_percent_in_states_unbounded(wolfram_cmm, state_ids, wolfram_session))
# convert_wolfram_number(get_percent_in_states_unbounded(wolfram_cmm, [1], wolfram_session))


In [ ]:
cmm = get_cmm(3, conservative_failure_params)

state_ids = get_live_state_ids(cmm, 3)
wolfram_cmm = get_wolfram_markov_model(cmm)

probability_dist = wl.PDF(wolfram_cmm(wl.t), state_ids)
total_probability = wl.Apply(wl.Plus, probability_dist)
integrated_probability = wl.Integrate(total_probability, [wl.t, 0, 100])

In [ ]:
wolfram_session.evaluate(integrated_probability)

In [ ]:
wolfram_session.evaluate(wolfram_cmm(wl.t))